# Improving Olist's Estimated Delivery Time (ETA)

Estimated Time Arrival (ETA) is a key component of modern e-commerce services. It allows to manage customer expectations by displaying a predicted date for customer delivery.

In this section we build a predictive model to improve Olist ETA prediction of orders `wait_time`, and compare it with Olist's `expected_wait_time`

## Load and check `orders` dataset

In [1]:
# Load all orders from our training_data, including those non-delivered

In [2]:
# Check print all orders not delivered and conclude - should we care ?

## 1 - Baseline Model - Olist’s current ETA model

❓ What's the baseline we want to optimize?

The delta between Olist' `expected_wait_time` and the real `wait_time`

Plot their respective histogram, median difference and RMSE

## 2 - OLS multivariate model

Create an OLS model using the `statsmodel.api.OLS` method (not the `statsmodel.formula.api.ols`) this time

In [ ]:
import statsmodels.api as sm

### prepare dataset of features X

In [3]:
# Create target variable y (Serie) and feature X (DataFrame) without data leaks!

In [190]:
# standardize (z-normalize) each features of X to compare coefficients

In [5]:
# Add a constant to your model

### Run OLS with statsmodels

In [129]:
# Check prediction distributions

In [6]:
# Compute RMSE (i.e 𝛔(residuals))

In [ ]:
# Visualize analyse coefficients with  partial regression plots

In [ ]:
# Check residuals

## 3 - (optional) Can we add more Features?

We add here additional features from the product details (size, category etc..)

#### Add products details associated with each order (sum of product sizes, sum of weights etc...)

#### Run new OLS